In [0]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

!apt install git-lfs

  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
  Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
  Using cached pyarrow-15.0.2-cp310-cp310-manylinux_2_28_x86_64.whl (38.3 MB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached aiohttp-3.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached safetensors-0.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Using 

In [0]:
!git config --global user.email "aditi.kannan04@gmail.com"
!git config --global user.name "Aditi-Kannan"

In [0]:
from huggingface_hub import notebook_login

notebook_login()

In [0]:
from datasets import load_dataset
import datasets

In [0]:
from transformers import AutoTokenizer
model_checkpoint = "deepset/roberta-base-squad2"
#model_checkpoint = "aditi2212/roberta-finetuned-subjqa-ChennaiQA-expanded"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [0]:
#max_length = 384
#stride = 128
max_length = 500
stride = 30

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [0]:
from pyspark.sql import SparkSession
df_train1 = spark.read.table("myllmwrkspc.dbo.train_data_final")
df_test1 = spark.read.table("myllmwrkspc.dbo.test_data_final")


In [0]:
import pandas as pd
df_train = df_train1.toPandas()
df_test = df_test1.toPandas()

In [0]:
df_train=df_train[['question','human_ans_indices','review','human_ans_spans']]
df_test=df_test[['question','human_ans_indices','review','human_ans_spans']]

In [0]:
for ii in range(0,50):
    print(ii, df_train['human_ans_indices'].iloc[ii])


0 (27, 33)
1 (0, 7)
2 (189, 247)
3 (150, 161)
4 (104, 146)
5 (78, 88)
6 (304, 320)
7 (361, 406)
8 (67, 117)
9 (186, 220)
10 (236, 264)
11 (317, 365)
12 (403, 438)
13 (387, 476)
14 (134, 180)
15 (85, 93)
16 (268, 272)
17 (66, 135)
18 (32, 76)
19 (118, 177)
20 (270, 319)
21 (320, 415)
22 (284, 341)
23 (352, 433)
24 (434, 501)
25 (59, 71)
26 (88, 130)
27 (132, 180)
28 (185, 219)
29 (236, 240)
30 (484, 497)
31 (72, 103)
32 (354, 382)
33 (318, 387)
34 (389, 507)
35 (27, 51)
36 (376, 403)
37 (388, 444)
38 (471, 506)
39 (542, 597)
40 (84, 131)
41 (33, 53)
42 (228, 256)
43 (317, 339)
44 (48, 79)
45 (130, 177)
46 (255, 279)
47 (294, 310)
48 (312, 336)
49 (338, 361)


In [0]:
import numpy as np
df_train['id']=np.linspace(0,len(df_train)-1,len(df_train))
df_test['id']=np.linspace(0,len(df_test)-1,len(df_test))

df_train['id']=df_train['id'].astype(str)
df_test['id']=df_test['id'].astype(str)

In [0]:
int(df_train.iloc[5].human_ans_indices.split('(')[1].split(',')[0])

78

In [0]:
df_train.iloc[0].human_ans_indices

'(27, 33)'

In [0]:
float(df_train.iloc[0].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])

33.0

In [0]:
df_train['answers']=df_train['human_ans_spans']
df_test['answers']=df_test['human_ans_spans']

In [0]:
for i in range(0,len(df_train)):
  answer1={}
  si=int(df_train.iloc[i].human_ans_indices.split('(')[1].split(',')[0])
  ei=int(df_train.iloc[i].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])
  answer1['text']=[df_train.iloc[i].review[si:ei]]
  answer1['answer_start']=[si]
  df_train.at[i, 'answers']=answer1
  #print(df_train.iloc[i].answers,df_train.iloc[i].human_ans_spans)

In [0]:

for i in range(0,len(df_test)):
  answer1={}
  si=int(df_test.iloc[i].human_ans_indices.split('(')[1].split(',')[0])
  ei=int(df_test.iloc[i].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])
  answer1['text']=[df_test.iloc[i].review[si:ei]]
  answer1['answer_start']=[si]
  df_test.at[i, 'answers']=answer1
  #print(df_train.iloc[i].answers,df_train.iloc[i].human_ans_spans)

In [0]:
df_train.columns

Index(['question', 'human_ans_indices', 'review', 'human_ans_spans', 'id',
       'answers'],
      dtype='object')

In [0]:
df_train.head()

,question,human_ans_indices,review,human_ans_spans,id,answers
0,What is Chennai formerly known as?,"(27, 33)","Chennai, formerly known as Madras, is the capi...",Madras,0.0,"{'text': ['Madras'], 'answer_start': [27]}"
1,What is the capital city of Tamil Nadu?,"(0, 7)","Chennai, formerly known as Madras, is the capi...",Chennai,1.0,"{'text': ['Chennai'], 'answer_start': [0]}"
2,What is Chennai known for?,"(189, 247)","Chennai, formerly known as Madras, is the capi...","Its rich history, vibrant culture, and archite...",2.0,"{'text': ['rich history, vibrant culture, and ..."
3,In which region of India is Chennai located?,"(150, 161)","Chennai, formerly known as Madras, is the capi...",South India,3.0,"{'text': ['South India'], 'answer_start': [150]}"
4,What are some of the key features of Chennai?,"(104, 146)","Chennai, formerly known as Madras, is the capi...","Cultural, economic, and educational center",4.0,"{'text': ['cultural, economic, and educational..."


In [0]:
df_train.columns=['question', 'human_ans_indices', 'context', 'human_ans_spans', 'id',
       'answers']

df_test.columns=['question', 'human_ans_indices', 'context', 'human_ans_spans','id',
       'answers']

In [0]:
val_dataset2 = datasets.Dataset.from_pandas(df_test)
train_dataset2 = datasets.Dataset.from_pandas(df_train)


In [0]:
train_dataset = train_dataset2.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_dataset2.column_names,
)
len(train_dataset2), len(train_dataset)

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

(271, 271)

In [0]:
train_dataset2.shape

(271, 6)

In [0]:
train_dataset2[0]

{'question': 'What is Chennai formerly known as?',
 'human_ans_indices': '(27, 33)',
 'context': 'Chennai, formerly known as Madras, is the capital city of the Indian state of Tamil Nadu. It is a major cultural, economic, and educational center in South India. The city is known for its rich history, vibrant culture, and architectural landmarks. Chennai boasts a diverse range of tourist attractions. The Marina Beach, stretching along the Bay of Bengal, is one of the longest urban beaches in the world and a popular spot for locals and tourists alike.',
 'human_ans_spans': 'Madras',
 'id': '0.0',
 'answers': {'answer_start': [27], 'text': ['Madras']}}

In [0]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [0]:
validation_dataset = val_dataset2.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val_dataset2.column_names,
)
len(validation_dataset)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

49

In [0]:
val_dataset2[0]

{'question': 'Best time to visit Guindy National Park?',
 'human_ans_indices': '(126, 146)',
 'context': 'Guindy National Park can be visited anytime during the year however the favourite time of travellers visiting Chennai is from November to February when the temperatures are less as compared to the rest of the year. Major spectacles located in the vicinity of Guindy National Park are Birla Planetarium, Phoenix Marketcity, and MGR film city.  Address: Opp IIT Madras, Guindy National Park, Guindy, Chennai, Tamil Nadu, 600025, India. Phone: 044-29505150, +91 4422200335. E-Mail: wlwchennai@yahoo.co.in',
 'human_ans_spans': 'November to February',
 'id': '0.0',
 'answers': {'answer_start': [126], 'text': ['November to February']}}

In [0]:
validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 49
})

In [0]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 271
})

In [0]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [0]:
import torch
from transformers import AutoModelForQuestionAnswering


In [0]:
import collections



In [0]:
import evaluate

metric = evaluate.load("squad")

In [0]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [0]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [0]:
from transformers import TrainingArguments

args = TrainingArguments(
    "roberta-finetuned-ChennaiQA-10",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=False,
    push_to_hub=True,
)

In [0]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)


/local_disk0/.ephemeral_nfs/envs/pythonEnv-ba18ea09-3553-478d-9ed7-7687f8d5a012/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [0]:
import numpy as np
n_best=20
max_answer_length =500

In [0]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, val_dataset2)

  0%|          | 0/49 [00:00<?, ?it/s]

{'exact_match': 53.06122448979592, 'f1': 77.53121594958327}

In [0]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.200300,No log
2,0.680900,No log
3,0.429400,No log
4,0.290100,No log
5,0.223100,No log
6,0.129600,No log
7,0.098800,No log
8,0.080900,No log
9,0.060300,No log
10,0.051800,No log


Attempted to log scalar metric loss:
1.2003
Attempted to log scalar metric grad_norm:
43.12065505981445
Attempted to log scalar metric learning_rate:
1.8e-05
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_runtime:
20.2433
Attempted to log scalar metric eval_samples_per_second:
2.421
Attempted to log scalar metric eval_steps_per_second:
0.346
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric loss:
0.6809
Attempted to log scalar metric grad_norm:
25.209630966186523
Attempted to log scalar metric learning_rate:
1.6000000000000003e-05
Attempted to log scalar metric epoch:
2.0
Attempted to log scalar metric eval_runtime:
16.3283
Attempted to log scalar metric eval_samples_per_second:
3.001
Attempted to log scalar metric eval_steps_per_second:
0.429
Attempted to log scalar metric epoch:
2.0
Attempted to log scalar metric loss:
0.4294
Attempted to log scalar metric grad_norm:
24.94563102722168
Attempted to log scalar metric learning_rate:


TrainOutput(global_step=340, training_loss=0.32451727109796863, metrics={'train_runtime': 18163.8547, 'train_samples_per_second': 0.149, 'train_steps_per_second': 0.019, 'total_flos': 691517794549760.0, 'train_loss': 0.32451727109796863, 'epoch': 10.0})

In [0]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/aditi2212/roberta-finetuned-ChennaiQA-10/commit/7c111065436c74429532fddffdb030e0ca43d305', commit_message='Training complete', commit_description='', oid='7c111065436c74429532fddffdb030e0ca43d305', pr_url=None, pr_revision=None, pr_num=None)

In [0]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, val_dataset2)

  0%|          | 0/49 [00:00<?, ?it/s]

{'exact_match': 81.63265306122449, 'f1': 88.99386327957757}

In [0]:
#Inference!

In [0]:
from transformers import pipeline

In [0]:
!pip install tf-keras

  Using cached tf_keras-2.16.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Using cached h5py-3.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.36.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.1 MB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached keras-3.2.1-py3-none-any.whl (1.1 MB)
  Using cached ml_dtypes-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.wh

In [0]:
# Replace this with your own checkpoint
#model_checkpoint2 = "aditi2212/roberta-finetuned-subjqa-ChennaiQA-expanded"
model_checkpoint2 = "aditi2212/roberta-finetuned-ChennaiQA-10"
question_answerer = pipeline("question-answering", model=model_checkpoint2)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [0]:
import pandas as pd
# df_train1=pd.read_csv('/content/Train50.csv')
# df_test1=pd.read_csv('/content/Test24.csv')

from pyspark.sql import SparkSession
df_train2 = spark.read.table("myllmwrkspc.dbo.train_data_final")
df_test2 = spark.read.table("myllmwrkspc.dbo.test_data_final")

df_train22 = df_train2.toPandas()
df_test22 = df_test2.toPandas()


In [0]:
df_train22.iloc[35].question

'How many species of crocodiles can I see at madras croc bank?'

In [0]:
context = df_train22.iloc[35].review
question = df_train22.iloc[35].question
question_answerer(question=question, context=context)

{'score': 0.9992237091064453,
 'start': 27,
 'end': 51,
 'answer': '15 species of crocodiles'}

In [0]:
# Replace this with your own checkpoint
model_checkpoint_o = "deepset/roberta-base-squad2"
question_answerer_old = pipeline("question-answering", model=model_checkpoint_o)

In [0]:
context = df_train22.iloc[2].review
question = df_train22.iloc[2].question
question_answerer_old(question=question, context=context)

{'score': 0.4973326027393341,
 'start': 189,
 'end': 247,
 'answer': 'rich history, vibrant culture, and architectural landmarks'}

In [0]:
df_train.iloc[3].question

'In which region of India is Chennai located?'

In [0]:
df_train.iloc[3].answers

{'text': ['South India'], 'answer_start': [150]}

In [0]:
df_train[['id','question','context','answers']].head()

,id,question,context,answers
0,0.0,What is Chennai formerly known as?,"Chennai, formerly known as Madras, is the capi...","{'text': ['Madras'], 'answer_start': [27]}"
1,1.0,What is the capital city of Tamil Nadu?,"Chennai, formerly known as Madras, is the capi...","{'text': ['Chennai'], 'answer_start': [0]}"
2,2.0,What is Chennai known for?,"Chennai, formerly known as Madras, is the capi...","{'text': ['rich history, vibrant culture, and ..."
3,3.0,In which region of India is Chennai located?,"Chennai, formerly known as Madras, is the capi...","{'text': ['South India'], 'answer_start': [150]}"
4,4.0,What are some of the key features of Chennai?,"Chennai, formerly known as Madras, is the capi...","{'text': ['cultural, economic, and educational..."


In [0]:
df_train22

,qas_id,review,question,human_ans_spans,human_ans_indices
0,q1,"Chennai, formerly known as Madras, is the capi...",What is Chennai formerly known as?,Madras,"(27, 33)"
1,q2,"Chennai, formerly known as Madras, is the capi...",What is the capital city of Tamil Nadu?,Chennai,"(0, 7)"
2,q3,"Chennai, formerly known as Madras, is the capi...",What is Chennai known for?,"Its rich history, vibrant culture, and archite...","(189, 247)"
3,q4,"Chennai, formerly known as Madras, is the capi...",In which region of India is Chennai located?,South India,"(150, 161)"
4,q5,"Chennai, formerly known as Madras, is the capi...",What are some of the key features of Chennai?,"Cultural, economic, and educational center","(104, 146)"
...,...,...,...,...,...
266,q314,The Shore Temple is a complex of temples and s...,Where is shore temple?,Mahabalipuram,"(117, 130)"
267,q316,The Shore Temple is a complex of temples and s...,What is the shore temple built with?,blocks of granite,"(229, 246)"
268,q318,The Shore Temple is a complex of temples and s...,When was shore temple classified as UNESCO wor...,1984,"(392, 396)"
269,q319,The Shore Temple is a complex of temples and s...,What is unique about shore temple?,one of the oldest structural stone temples of ...,"(404, 464)"


In [0]:
context = df_train22.iloc[0].review
question_answerer(question='name some local food in chennai?', context=context)

{'score': 0.9964573979377747,
 'start': 304,
 'end': 320,
 'answer': 'The Marina Beach'}

In [0]:
df_test.head()

,question,human_ans_indices,context,human_ans_spans,id,answers
0,Best time to visit Guindy National Park?,"(126, 146)",Guindy National Park can be visited anytime du...,November to February,0.0,"{'text': ['November to February'], 'answer_sta..."
1,Why should I visit Guindy national park betwee...,"(156, 177)",Guindy National Park can be visited anytime du...,temperatures are less,1.0,"{'text': ['temperatures are less'], 'answer_st..."
2,Who operates vandalur zoo?,"(340, 370)","Arignar Anna Zoological Park (AAZP), commonly ...",Central Zoo Authority of India,2.0,"{'text': ['Central Zoo Authority of India'], '..."
3,What area does vandalur zoo span?,"(402, 414)","Arignar Anna Zoological Park (AAZP), commonly ...",602 hectares,3.0,"{'text': ['602 hectares'], 'answer_start': [402]}"
4,Which is India's largest zoo?,"(0, 28)",Arignar Anna Zoological Park is India's larges...,Arignar Anna Zoological Park,4.0,"{'text': ['Arignar Anna Zoological Park'], 'an..."


In [0]:
frames = [df_train, df_test]
df_total = pd.concat(frames)

unique_context = pd.DataFrame(df_total['context'].unique())
unique_context.rename( columns={0:'Context'}, inplace=True )
contexts_list = unique_context['Context'].tolist()


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example question
question = input()

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()


# Fit and transform the contexts
tfidf_matrix = tfidf_vectorizer.fit_transform(contexts_list)

# Transform the question using the same vectorizer
question_tfidf = tfidf_vectorizer.transform([question])

# Compute cosine similarity between question and each context
similarity_scores = cosine_similarity(question_tfidf, tfidf_matrix)

# Get the index of the context with the highest similarity score
most_similar_index = similarity_scores.argmax()

# Print the most relevant context and its relevance score
print("Most relevant context:", contexts_list[most_similar_index])
print("Relevance score:", similarity_scores[0][most_similar_index])
question_answerer(question=question, context=contexts_list[most_similar_index])


 famous buildings in chennai?

Most relevant context: As Kovalam beach is near Chennai and is located on the way to Mahabalipuram, it is visited by thousands of people each day. Mostly the local crowd is higher in the beach. Fishing is the main occupation of the people here. Though the beach is famous, it is littered by tourists and the local people alike. Along with the beach, the village of Covelong, famous for its fishing activities draws many tourists. It is India's first surfing village. There are surfing schools here.
Relevance score: 0.14608916675080014


{'score': 0.23405838012695312,
 'start': 331,
 'end': 350,
 'answer': 'village of Covelong'}